## Importing the Libraries

In [ ]:
import pandas as pd,numpy as np
import seaborn as sns,matplotlib.pyplot as plt
%matplotlib inline


## Importing the daata for project

In [ ]:
df = pd.read_excel('marketing_campaign.xlsx')

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.columns

## Setting Display option to ensure feature name visibility

In [ ]:
pd.set_option('display.max_columns',None)

## Warning Supperssion

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Since Id feature wont be usefull to train our model.So dropping Id column

In [ ]:
df = df.drop(['ID'],axis =1)

## Defining Target and Independent Feature

In [ ]:
x = df.drop(['Response'],axis =1)
y= df[['Response']]

## Get the Response Rate

In [ ]:
y.mean()

### So Lets start Exploring the Independent Features

#### Here are two columns with year_birth and dt_customer with dates values

## Customer age and Tenure

In [ ]:
import datetime
reference_date  = datetime.datetime(2022,1,1)
print(reference_date)

In [ ]:
x['Year_Birth'] = pd.to_datetime(df['Year_Birth'],format = '%Y')
x['Age'] = (reference_date-x['Year_Birth']).astype('timedelta64[Y]')
x['Age'].head()

In [ ]:
x['Dt_Customer'] = pd.to_datetime(x['Dt_Customer'])
x['Customer_Tenure'] = (reference_date-x['Dt_Customer']).astype('timedelta64[Y]')
x['Customer_Tenure'].head()

### We get the age and customer tenure.So we can drop year_birth and dt_customer columns now

In [ ]:
x=x.drop(['Year_Birth','Dt_Customer'],axis = 1)

## Spliting Features into Numerical and Categorical

In [ ]:
x_num = x.select_dtypes(include= 'number')
x_char = x.select_dtypes(include = 'object')

In [ ]:
print('The shape of Numerical Columns is ',x_num.shape)
print('The shape of Categorical Columns is',x_char.shape)

In [ ]:
def unique_values(x):
    x=x.value_counts().count()
    return(x)
df_value_counts = pd.DataFrame(x_num.apply(lambda x: unique_values(x)))

In [ ]:
df_value_counts.columns = ['feature_Counts']                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [ ]:
df_value_counts

### So here we can see some columns with less unique values.These can considered to be categorical for practile purpose

In [ ]:
# let establish the threshold 

slice1 =df_value_counts.loc[df_value_counts['feature_Counts']<= 20]
cat_list=slice1.index
cat  =x_num.loc[:,cat_list]
cat.dtypes

In [ ]:
def data_type_conv(x):
    x=x.astype('object')
    return(x)
cat = cat.apply(lambda x:data_type_conv(x))
cat.dtypes

In [ ]:
slice2 = df_value_counts.loc[df_value_counts['feature_Counts']>20]
num_list = slice2.index
x_num =x_num.loc[:,num_list]
x_num

In [ ]:
x_char = pd.concat([x_char,cat],axis=1,join= 'inner')

In [ ]:
print('The new shape of numerical columns is',x_num.shape)

In [ ]:
print('The new Shape of Categorical columns is',x_char.shape)

## Outlier Analysis of Numerical Features

In [ ]:
x_num.describe(percentiles= [0.01,0.05,0.10,0.15,0.25,0.50,0.75,0.85,0.90,0.99])

## Capping and Flooring of Outliers

In [ ]:
def outlier(x):
    x = x.clip(lower= x.quantile(0.01))
    x = x.clip(upper = x.quantile(0.99))
    return(x)

In [ ]:
x_num = x_num.apply(lambda x: outlier(x))

In [ ]:
x_num.describe(percentiles= [0.01,0.05,0.10,0.15,0.25,0.50,0.75,0.85,0.90,0.99])

## Missing Value Handling - Numerical Features

In [ ]:
x_num.isnull().sum()

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values= np.nan,strategy ='mean')
x_num1 = pd.DataFrame(imputer.fit_transform(x_num),index = x_num.index,columns=x_num.columns)

In [ ]:
x_num1.isnull().mean()

## Missing Value Handling - Categorical Features

In [ ]:
x_char.isnull().mean()

## Feature Selection-Numerical Features

### 1- Remove Features with 0 Variance

In [ ]:
from sklearn.feature_selection import VarianceThreshold
vt = VarianceThreshold(threshold = 0)

vt.fit_transform(x_num1)
cols = vt.get_support(indices = True)
x_num2 = x_num1.iloc[:,cols]

In [ ]:
x_num2.iloc[0]

### 2- Bi Variate Analysis 

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer
discrete = KBinsDiscretizer(n_bins= 10,encode= 'ordinal',strategy ='quantile')
x_num_bin = pd.DataFrame(discrete.fit_transform(x_num2),index = x_num2.index,columns = x_num2.columns).add_suffix('_rank')
x_num_bin

In [ ]:
x_bin_combined = pd.concat([y,x_num_bin],axis = 1,join='inner')
x_bin_combined=x_bin_combined.astype(int)
x_bin_combined.info()


In [ ]:
x_bin_comdined = x_bin_combined.astype(int)
from numpy import mean
for col in (x_num_bin.columns):
    plt.figure()
    sns.lineplot(x=col,y=x_bin_combined['Response'].mean(),data = x_bin_combined,color = 'red')
    sns.barplot(x=col,y='Response',data = x_bin_combined,estimator = mean)
plt.show()    

In [ ]:
x_num2  = x_num2.drop(['Age'],axis=1)

In [ ]:
x_num2.shape

### 3- Select K Best

In [ ]:
from sklearn.feature_selection import SelectKBest,chi2
selector = SelectKBest(chi2,k=6)
selector.fit_transform(x_num2,y)
cols = selector.get_support(indices = True)
select_features_df_num  = x_num1.iloc[:,cols]

In [ ]:
select_features_df_num 

## Feature Selection - Categorical Features

In [ ]:
char_unique_count = pd.DataFrame(x_char.apply(lambda x: unique_values(x)))
char_unique_count

In [ ]:
char_unique_count.columns = ['feature_levels']
slice1 = char_unique_count.loc[char_unique_count['feature_levels']>1]
cat_list  = slice1.index
x_char = x_char.loc[:,cat_list]
x_char

In [ ]:
x_char_merged= pd.concat([y,x_char],axis =1,join = 'inner')
from numpy import mean
for col in (x_char.columns):
    plt.figure()
    sns.lineplot(x=col,y=x_char_merged['Response'].mean(),data = x_char_merged,color = 'red')
    sns.barplot(x=col,y='Response',data = x_char_merged,estimator = mean)
plt.show()    

In [ ]:
##dropping complain cloumn
x_char = x_char.drop(['Complain'],axis = 1)

## Create dummy Features with n-1 levels`

In [ ]:
x_char_dum = pd.get_dummies(x_char,drop_first = True)
x_char_dum.shape

## 2- Select K Best

In [ ]:
from sklearn.feature_selection import SelectKBest,chi2
selector = SelectKBest(chi2,k=85)
selector.fit_transform(x_char_dum,y)
#get columns to keep and create new dataframe with these only
cols = selector.get_support(indices = True)
select_features_df_char = x_char_dum.iloc[:,cols]

In [ ]:
select_features_df_char.dtypes

## Creting the master feature set for model development

In [ ]:
x_all = pd.concat([select_features_df_char,select_features_df_num ],axis = 1,join ='inner')

In [ ]:
x_all

##  Dimensionality Reduction throug Variable Clustering

In [ ]:
from varclushi import VarClusHi
vc= VarClusHi(x_all,maxeigval2=1,maxclus = None)
vc.varclus()

In [ ]:
vc.info

In [ ]:
check =vc.rsquare
check

In [ ]:
filter =check[check['RS_Ratio']<= 0.5]
filter.shape

In [ ]:
final_features = filter['Variable']
x_all = x_all[final_features]
x_all.shape

## Train_Test_split

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test  = train_test_split(x_all,y,test_size = 0.3,random_state = 20)

In [ ]:
print('Shape of Training Data',x_train.shape)
print('Shape of Testing Data',x_test.shape)
print('Shape of response Training Data',y_train.shape)
print('Shape of response Testing Data',y_test.shape)


# Building the models

## Building the LogisticRegression Model

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(x_train,y_train)

## Building the Decision Tree Model

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier(criterion = 'gini',random_state =20)

## Grid Search for model Optimization

In [ ]:
from sklearn.model_selection import GridSearchCV
param_dist = {'max_depth':[3,4,5,6],'min_samples_split':[50,100,125,150]}
tree_grid = GridSearchCV(dtree,cv = 10,param_grid= param_dist,n_jobs =-1)
tree_grid.fit(x_train,y_train)
print('Best parameters using grid search: \n',tree_grid.best_params_)

In [ ]:
dtree = DecisionTreeClassifier(criterion = 'gini',max_depth=3,min_samples_split= 50,random_state =20)
dtree.fit(x_train,y_train)

## Building the RandomForest Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(criterion ='gini',max_depth = 3,min_samples_split = 50,random_state =20)
rf.fit(x_train,y_train)

## Building the Gradient Boosting Model

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbm = GradientBoostingClassifier(criterion = 'mse',random_state = 20,max_depth = 3,min_samples_split = 50)
gbm.fit(x_train,y_train)

## Model Evaluation

In [ ]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix

In [ ]:
y_pred_logre= logreg.predict(x_test)
y_pred_tree = dtree.predict(x_test)
y_pred_rf = rf.predict(x_test)
y_pred_gbm = gbm.predict(x_test)


In [ ]:
print('Accuracy from Logistic Regression Mode:',metrics.accuracy_score(y_test,y_pred_logre))
print('Precision from Logistic Regression Model:',metrics.precision_score(y_test,y_pred_logre))
print('Recall from Logistic Regression Model:',metrics.recall_score(y_test,y_pred_logre))
print('f1_score from Logistic Regression Model:',metrics.f1_score(y_test,y_pred_logre))
print('Area under ROC Curve from Logistic Regression Model:',metrics.roc_auc_score(y_test,y_pred_logre))

In [ ]:
metrics.plot_confusion_matrix(logreg,x_all,y)

In [ ]:
print('Accuracy from Decision Tree  Mode:',metrics.accuracy_score(y_test,y_pred_tree))
print('Precision from  Decision Tree Model:',metrics.precision_score(y_test,y_pred_tree))
print('Recall from Decision Tree Model:',metrics.recall_score(y_test,y_pred_tree))
print('f1_score from Decision Tree Model:',metrics.f1_score(y_test,y_pred_tree))
print('Area under ROC Curve from Decision Tree Model:',metrics.roc_auc_score(y_test,y_pred_tree))

In [ ]:
metrics.plot_confusion_matrix(dtree,x_all,y)

In [ ]:
print('Accuracy from Random forest Model:',metrics.accuracy_score(y_test,y_pred_rf))
print('Precision from  Random forest Model:',metrics.precision_score(y_test,y_pred_rf))
print('Recall from Random forest Model:',metrics.recall_score(y_test,y_pred_rf))
print('f1_score from Random forest Model:',metrics.f1_score(y_test,y_pred_rf))
print('Area under ROC Curve from Random forest Model:',metrics.roc_auc_score(y_test,y_pred_rf))

In [ ]:
metrics.plot_confusion_matrix(rf,x_all,y)

In [ ]:
print('Accuracy from Gradient Boosting Model:',metrics.accuracy_score(y_test,y_pred_gbm))
print('Precision from  Gradient Boosting Model:',metrics.precision_score(y_test,y_pred_gbm))
print('Recall from Gradient Boosting Model:',metrics.recall_score(y_test,y_pred_gbm))
print('f1_score from Gradient Boosting Model:',metrics.f1_score(y_test,y_pred_gbm))
print('Area under ROC Curve from Gradient Boosting Model:',metrics.roc_auc_score(y_test,y_pred_gbm))

In [ ]:
metrics.plot_confusion_matrix(gbm,x_all,y)

## Extracting Feature Importance from the best model

In [ ]:
feature_importances = pd.DataFrame(gbm.feature_importances_,
                                  index =x_train.columns,
                                  columns= ['importance']).sort_values('importance',ascending = False)
feature_importances.shape

## Extracting the Top 10 important model features

In [ ]:
feature_importances.iloc[0:10]

## Plotting the top 10 feature importance in a Horizontal Bar Plot

In [ ]:
barp = sns.barplot(x='importance',y= feature_importances.iloc[0:10].index,data=feature_importances.iloc[0:10])

## Probability Decile Analysis and Gains Chart/Lorenz Curve

### 1- Getting the Model Probabilities

In [ ]:
y_pred_prob = gbm.predict_proba(x_all)[:,1]
df['pred_prob'] = pd.DataFrame(y_pred_prob)

### 2- Create Deciles based on the model probabilities

In [ ]:
df['P_Rank_gbm'] = pd.qcut(df['pred_prob'].rank(method ='first').values,10,duplicates = 'drop').codes+1
df[['pred_prob','P_Rank_gbm']].head(10)

### 3-  Summarize the data at decile Level for Analysis

In [ ]:
rank_df = df.groupby('P_Rank_gbm')['Response'].agg(['count','mean']).sort_values(by = 'P_Rank_gbm',ascending =False)
rank_df.rename(columns = {'mean':'Actual_event_rate'},inplace =True)
rank_df

### 4 - Event Capture Analysis across probabilities Deciles

In [ ]:
rank_df['N_events'] = rank_df['count']*rank_df['Actual_event_rate']
rank_df['cum_events'] = rank_df['N_events'].cumsum()
rank_df['event_cap'] = rank_df['N_events']/max(rank_df['N_events'].cumsum())
rank_df['cum_event_cap'] = rank_df['event_cap'].cumsum()
rank_df

### 5- Non event Capture Analysis across Probability Deciles

In [ ]:
rank_df['N_non_events']  = rank_df['count']- rank_df['N_events']
rank_df['cum_non_events'] = rank_df['N_non_events'].cumsum()
rank_df['non_event_cap'] = rank_df['N_non_events']/max(rank_df['N_non_events'].cumsum())
rank_df['cum_non_event_cap'] = rank_df['non_event_cap'].cumsum()
rank_df

### 6- Compute KS Statistic and lift over Baseline Event Rate

In [ ]:
rank_df['KS'] = round((rank_df['cum_event_cap'] - rank_df['cum_non_event_cap']),4)
rank_df['Baseline_Event_Rate'] = (max(rank_df['cum_events'])/max(rank_df['count'].cumsum()))
rank_df['Lift_over_avg'] = rank_df['Actual_event_rate']/rank_df['Baseline_Event_Rate']
rank_df

### 7 - Create Deciles(1-10) column to indicate the Decile Number

In [ ]:
rank_df= rank_df.reset_index()
rank_df['Decile'] = rank_df.index+1
rank_df

## Visualization the performance of model probabilities across deciles

### 8 - Create the rank ordering chart

In [ ]:
ax = sns.lineplot(x="Decile", y="Actual_event_rate", data=rank_df,color='green').set_title('Rank Ordering Chart')
ax = sns.scatterplot( x="Decile", y="Baseline_Event_Rate", data=rank_df,color='red')
ax.legend(['Actual_event_rate', 'Baseline_Event_Rate'])

### 9- Create the lift chart

In [ ]:
ax = sns.barplot( x="Decile", y="Lift_over_avg", data=rank_df,color='blue').set_title('Lift Chart')

### 10- Create the Gains Chart/ Lorenz Curve

In [ ]:
ax = sns.lineplot( x="Decile", y="cum_event_cap", data=rank_df,color='red').set_title('Rank Ordering Chart')
ax = sns.lineplot( x="Decile", y="cum_non_event_cap", data=rank_df,color='green')
ax.legend(['cum_event_cap', 'cum_non_event_cap'])

## Is our model able to maximise event/non-event discrimination by the top 3 deciles?

In [ ]:
ax = sns.barplot( x="Decile", y="KS", data=rank_df,color='brown')
ax.legend(['KS'])

## Utlize APT framework to build a Machine Learning Driven Strategy

### A - Audience (Who do we traget?)

In [ ]:
df['Predicted_Resp_Rank'] = np.where(df['P_Rank_gbm']>=9,'Top 2','Bottom 8')
df['Predicted_Resp_Rank'].value_counts()

In [ ]:
df_top2 = df.loc[df['Predicted_Resp_Rank'] == 'Top 2',:]
df_top2.shape

### P - Prioritization (Based on Bussiness Value and Customer Engagement)

In [ ]:
df_top2['spend_count'] = np.count_nonzero(df_top2[['MntMeatProducts','MntWines','MntFruits','MntFishProducts','MntSweetProducts','MntGoldProds']],axis =1)
df_top2['spend_count'].describe()

In [ ]:
df_top2['Engagement'] = np.where(df_top2['spend_count']< df_top2['spend_count'].quantile(0.75),'Low','High')
df_top2['Engagement'].value_counts()

In [ ]:
df_top2['Total_Spend']=df_top2['MntWines']+df_top2['MntFruits']+df_top2['MntMeatProducts']+df_top2['MntFishProducts']+df_top2['MntSweetProducts']+df_top2['MntGoldProds']
df_top2['Total_Spend'].describe()

In [ ]:
df_top2['spend_level']=np.where(df_top2['Total_Spend']<df_top2['Total_Spend'].quantile(0.75),"Low Spend","High Spend")
df_top2['spend_level'].value_counts()

In [ ]:
pd.crosstab(index=df_top2['spend_level'], columns=df_top2['Engagement'],values=df_top2['Response'],aggfunc='mean')

In [ ]:
pd.crosstab(index=df_top2['spend_level'], columns=df_top2['Engagement'],values=df_top2['Response'],aggfunc='count')

In [ ]:
df_priority_1=df_top2.loc[(df_top2['spend_level']=='High Spend') & (df_top2['Engagement']=='Low'),:]
df_priority_1.shape

In [ ]:
df_priority_2=df_top2.loc[(df_top2['spend_level']=='High Spend') & (df_top2['Engagement']=='High'),:]
df_priority_2.shape

In [ ]:
df_priority_3=df_top2.loc[(df_top2['spend_level']=='Low Spend') & (df_top2['Engagement']=='High'),:]
df_priority_3.shape

In [ ]:
df_priority_4=df_top2.loc[(df_top2['spend_level']=='Low Spend') & (df_top2['Engagement']=='Low'),:]
df_priority_4.shape

## T - Treatment(Identify Service/Product Preferences of each Priority Segment)

In [ ]:
spend_cat_priority1=df_priority_1[['MntWines','MntFruits','MntMeatProducts','MntFishProducts','MntSweetProducts','MntGoldProds']]

In [ ]:
spend_avg = spend_cat_priority1.mean(axis=0).sort_values(ascending=False)
spend_avg

In [ ]:
spend_cat_priority2=df_priority_2[['MntWines','MntFruits','MntMeatProducts','MntFishProducts','MntSweetProducts','MntGoldProds']]

In [ ]:
spend_avg = spend_cat_priority2.mean(axis=0).sort_values(ascending=False)
spend_avg

In [ ]:
spend_cat_priority3=df_priority_3[['MntWines','MntFruits','MntMeatProducts','MntFishProducts','MntSweetProducts','MntGoldProds']]

In [ ]:
spend_avg = spend_cat_priority3.mean(axis=0).sort_values(ascending=False)
spend_avg

In [ ]:
spend_cat_priority4=df_priority_4[['MntWines','MntFruits','MntMeatProducts','MntFishProducts','MntSweetProducts','MntGoldProds']]

In [ ]:
spend_avg = spend_cat_priority4.mean(axis=0).sort_values(ascending=False)
spend_avg

# Project Report

# Objective of the Project
Leverage the spend behavior and past campaign responses to build a response model that can
be used to devise a strategy that enhances the response rates and improves profitability of marketing campaigns

# Algorithms Used :
In this project we have used Logistic Regression, Decision Trees, Random Forests and Gradient Boosting Algorithms

# Final Model Algorithm
Among the models that we tried building the GBM Algorithm performed the best in terms of F1_Score and Area under ROC Curve.
Therefore we have kept GBM as the final model algorithm

# Model Performance Measures
Accuracy - 0.88

Precision - 0.75

Recall - 0.28

F1 Score - 0.41

AUC - 0.63

        

# Top 10 drivers from the Model

- MntMeatProducts
- Income
- AcceptedCmp3_1
- Customer_Tenure_9
- MntFishProducts
- NumStorePurchases_2
- Marital_Status_Together
- NumCatalogPurchases_10
- NumCatalogPurchases_6
- NumWebPurchases_7

# The Best Offers should be provided on the following products
- Wines Products
- Fish Products
- Meat Products
- Gold Products